# Labboration statistiska metoder

### Introduktion
I denna labb kommer jag att göra en linjär regression på ett dataset och sedan tolka datan statistiskt. Jag kommer bygga 2 stycken modeller, en med alla features och en modell med reducerade features.
I båda modellerna kommer den kategoriska datan läggas till i efterhand med och sedan kommer modellerna jämföras.

### Metod
Jag har skapat en regressionsklass i LinearRegression.py som hanterar alla min beräkningar. Jag ska undersöka datasetet "housing.csv" som är ett dataset innehållande huspriser från California 1997.
Jag kommer använda klassen på hela datasetet förutom det kategoriska då jag kommer göra en "one-hot-encode" och lägga till på båda modellerna i slutet. 

Vad vi kan se är att datasetet har 10 kolumner och 20640 rader. 9 av kolumnerna är numeriska och den enda kategoriska kolumnen är  "ocean_proximity".

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from data_utils import load_data
from LinearRegression import LinearRegression

#data = load_data("../Data/housing.csv")
df = pd.read_csv("../Data/housing.csv")

df.shape, list(enumerate(df.columns)), df.dtypes,


((20640, 10),
 [(0, 'longitude'),
  (1, 'latitude'),
  (2, 'housing_median_age'),
  (3, 'total_rooms'),
  (4, 'total_bedrooms'),
  (5, 'population'),
  (6, 'households'),
  (7, 'median_income'),
  (8, 'median_house_value'),
  (9, 'ocean_proximity')],
 longitude             float64
 latitude              float64
 housing_median_age    float64
 total_rooms           float64
 total_bedrooms        float64
 population            float64
 households            float64
 median_income         float64
 median_house_value    float64
 ocean_proximity        object
 dtype: object)

De enda NaN värderna i datasetet finns i kolumnen: "total_bedrooms".

In [2]:
df.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

Jag droppar alla NaN värderna och får 207 mindre observationer att använda i min regressionanalys. Vi kan även här se de olika kategoriska värdena samt deras frekvens.

In [3]:
df_clean = df.dropna()
df.shape, df_clean.shape, df_clean["ocean_proximity"].value_counts()

((20640, 10),
 (20433, 10),
 ocean_proximity
 <1H OCEAN     9034
 INLAND        6496
 NEAR OCEAN    2628
 NEAR BAY      2270
 ISLAND           5
 Name: count, dtype: int64)

I regressionsanalysen valdes "median_house_value" som målvariablen (y) och de andra 8 numeriska kolumnerna som förklarande variablar i modellen. Jag tar ej med den kategoriska variabeln, utan den kommer jag att "one-hot-encoda" för att lägga till i modellen efter.

In [4]:
X = df_clean.iloc[:, :8].to_numpy()
y = df_clean.iloc[:, 8].to_numpy()

X.shape, y.shape

((20433, 8), (20433,))

Jag använder min pearson metod för att ta fram korrelationen mellan mina förklarande variabler samt lägger till kolumnnamnet för att kunna se vilka kolumner som är vilka.

In [5]:
model = LinearRegression()
model.fit(X, y)

pearson_corr = model.pearson()
feature_names = df_clean.columns[:8]
threshold = 0.8

[
    (feature_names[i], feature_names[j], pearson_corr[i, j])
    for i in range(len(feature_names))
    for j in range(i + 1, len(feature_names))
    if abs(pearson_corr[i, j]) > threshold
]

[('longitude', 'latitude', np.float64(-0.9246161131160107)),
 ('total_rooms', 'total_bedrooms', np.float64(0.9303795046865075)),
 ('total_rooms', 'population', np.float64(0.8572812510982971)),
 ('total_rooms', 'households', np.float64(0.9189915343453183)),
 ('total_bedrooms', 'population', np.float64(0.8777467431529307)),
 ('total_bedrooms', 'households', np.float64(0.9797282708045647)),
 ('population', 'households', np.float64(0.9071859001745))]

### Features
I tabellen ovan ser vi att "total_rooms, "total_bedrooms, "population", och "households" har starka inbördes korrelation till varandra vilket indikerar att de alla visar samma överlappande information i datan.
Jag väljer därför att droppa "total_bedrooms och "households" och behåller "population" samt "total_rooms". Trots 86% kolinjäritet så behöll jag dessa 2 features för att de visar olika delar av bostadsmarknaden. "Population" är ett bra representativt mått för områdets efterfrågan, "total_rooms" beskriver områdets utbud.

Vi kan även se att "longitude" och "latitude" har en stark negativ korrelation med varandra. Det är ganska uppenbart då de är koordinater på öst-väst och nord-syd är nästan ortogonala med varandra. Jag behåller dessa 2 för att dom är viktiga att kunna beskriva geografiska läget i modellen. Vi ska komma ihåg att inom fastighetsbranschen är det location, location, location som gäller.

In [6]:
model_full = LinearRegression()
model_full.fit(X, y)


(model_full.n, model_full.d, model_full.df)

(20433, 8, 20424)

### Modell 1
innehåller 20433 observationer, 8 features och 20424 frihetsgrader. Förklaringsgrad (r2) på modellen är ca 64% och det genomsnittliga prediktionsfelet (RMSE) är ca 70 000.
RMSE 


In [7]:
summary_full = model_full.summary()
summary_full

{'n': 20433,
 'd': 8,
 'df': 20424,
 'R2': np.float64(0.6369116857335633),
 'SSE': np.float64(98856034611384.45),
 'Variance': np.float64(4840189708.743853),
 'RMSE': np.float64(69556.14839566677),
 'F-statistic': np.float64(4478.347194849657),
 'F p-value': np.float64(0.0)}

In [12]:
feature_names = ["Intercept"] + list(df_clean.columns[:8])

params_full = pd.DataFrame({
    "Parameter": feature_names,
    "Beta": model_full.beta,
    "Std.error": np.sqrt(np.diag(model_full.covariance_matrix())),
    "t-värde": model_full.t_statistic(),
    "p-värde": model_full.t_p_values(),
    "CI nedre": model_full.confidence_intervals()[:, 0],
    "CI övre": model_full.confidence_intervals()[:, 1],
})

params_full

,Parameter,Beta,Std.error,t-värde,p-värde,CI nedre,CI övre
0,Intercept,-3.585396e+06,62900.542834,-57.001030,0.000000e+00,-3.708686e+06,-3.462106e+06
1,longitude,-4.273012e+04,717.086966,-59.588477,0.000000e+00,-4.413567e+04,-4.132457e+04
2,latitude,-4.250974e+04,676.951557,-62.795833,0.000000e+00,-4.383662e+04,-4.118286e+04
3,housing_median_age,1.157900e+03,43.388597,26.686742,2.946266e-154,1.072855e+03,1.242945e+03
4,total_rooms,-8.249725e+00,0.794261,-10.386672,3.294848e-25,-9.806540e+00,-6.692910e+00
5,total_bedrooms,1.138207e+02,6.930592,16.422942,3.188906e-60,1.002362e+02,1.274052e+02
6,population,-3.838558e+01,1.084121,-35.407090,1.459679e-266,-4.051054e+01,-3.626061e+01
7,households,4.770135e+01,7.546555,6.320944,2.653505e-10,3.290950e+01,6.249320e+01
8,median_income,4.029752e+04,337.207172,119.503750,0.000000e+00,3.963657e+04,4.095847e+04


In [13]:
keep_idx = [0,1,2,3,5,7]
X_reduced = X[:, keep_idx]

model_reduced =LinearRegression()
model_reduced.fit(X_reduced, y)

summary_reduced = model_reduced.summary()
summary_reduced

{'n': 20433,
 'd': 6,
 'df': 20426,
 'R2': np.float64(0.6112645452966972),
 'SSE': np.float64(105838838802789.53),
 'Variance': np.float64(5181574405.306449),
 'RMSE': np.float64(71970.82246025889),
 'F-statistic': np.float64(5353.12187725878),
 'F p-value': np.float64(0.0)}

In [ ]:
feature_names = ["Intercept"] + list(df_clean.columns[:8])

params_reduced = pd.DataFrame({
    "Parameter": feature_names,
    "Beta": model_reduced.beta,
    "Std.error": np.sqrt(np.diag(model_reduced.covariance_matrix())),
    "t-värde": model_reduced.t_statistic(),
    "p-värde": model_reduced.t_p_values(),
    "CI nedre": model_reduced.confidence_intervals()[:, 0],
    "CI övre": model_reduced.confidence_intervals()[:, 1],
})

params_reduced

### Model 3
Detta är modell 2 fast med "one-hot-encoding" på kategoriska datan tillagda som features.

In [ ]:
ocean_dummies = pd.get_dummies(df_clean["ocean_proximity"], drop_first=True)
X_reduced_cat = np.column_stack([X_reduced, ocean_dummies.to_numpy()])


num_feature_names = df_clean.columns[:8].to_numpy()[keep_idx]
cat_feature_names = ocean_dummies.columns.to_numpy()

feature_names_reduced_cat = list(num_feature_names) + list(cat_feature_names)

model_reduced_cat = LinearRegression()
model_reduced_cat.fit(X_reduced_cat, y)

model_reduced_cat.summary()

{'n': 20433,
 'd': 10,
 'df': 20422,
 'R2': np.float64(0.6249671736790186),
 'SSE': np.float64(102108100433072.44),
 'Variance': np.float64(4999906984.285204),
 'RMSE': np.float64(70690.98464576689),
 'F-statistic': np.float64(3403.1899943471367),
 'F p-value': np.float64(0.0)}